### 1. 数据预处理和划分
1. 首先，确保有一个中文分词标注数据集。
2. 这个数据集应该包括已经分好词的句子，每个词用空格隔开。例如：
3. 去除标点符号。
4. 训练集/测试集比率为7:3
```bash
我 爱 北京 天安门
```

In [34]:
import re
import collections
import random


# 加载数据集并去除标点符号，确保每个句子至少有一个字
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    cleaned_lines = [re.sub(r'[^\w\s]', '', line).strip() for line in lines]
    # 过滤掉空行
    cleaned_lines = [line for line in cleaned_lines if len(line) > 0]
    return cleaned_lines

# 将数据集划分为训练集和测试集（70% : 30%）
def split_data(lines, train_ratio=0.7):
    random.shuffle(lines)
    split_point = int(len(lines) * train_ratio)
    return lines[:split_point], lines[split_point:]

# 数据集文件路径
file_path = 'RenMinData.txt'
lines = load_data(file_path)
train_lines, test_lines = split_data(lines)

# 构建词频词典
def build_vocab(lines):
    word_freq = collections.Counter()
    for line in lines:
        words = line.strip().split()
        word_freq.update(words)
    return word_freq

vocab = build_vocab(train_lines)
sorted_vocab = sorted(vocab.items(), key=lambda x: x[1], reverse=True)
print(len(sorted_vocab))
print(sorted_vocab[:5])


24019
[('的', 99718), ('了', 22500), ('在', 22415), ('和', 20074), ('是', 17908)]


## 2. 构建HMM模型
1. 定义状态和观测值
2. 计算初始概率、转移概率和发射概率

In [35]:
# 定义状态
states = ['B', 'M', 'E', 'S']

# 初始化概率矩阵
initial_prob = {state: 0 for state in states}
transition_prob = {state: {state_: 0 for state_ in states} for state in states}
emission_prob = {state: collections.Counter() for state in states}

# 统计频率并训练 HMM 模型
def train_hmm(lines):
    for line in lines:
        words = line.strip().split()
        if not words:
            continue
        line_states = []
        for word in words:
            if len(word) == 1:
                line_states.append('S')
            else:
                line_states.extend(['B'] + ['M'] * (len(word) - 2) + ['E'])
        
        initial_prob[line_states[0]] += 1
        
        for i in range(len(line_states) - 1):
            transition_prob[line_states[i]][line_states[i + 1]] += 1
        
        for word, state_seq in zip(words, line_states):
            for char, state in zip(word, state_seq):
                emission_prob[state][char] += 1

train_hmm(train_lines)

# 转换为概率
initial_prob = {state: count / sum(initial_prob.values()) for state, count in initial_prob.items()}
transition_prob = {state: {state_: count / sum(transition_prob[state].values()) for state_, count in state_dict.items()} for state, state_dict in transition_prob.items()}
emission_prob = {state: {char: count / sum(emission_prob[state].values()) for char, count in char_dict.items()} for state, char_dict in emission_prob.items()}


In [36]:
import json
print("initial_prob: ")
print(json.dumps(initial_prob, indent=2))

print("transition_prob: ")
print(json.dumps(transition_prob, indent=2))

initial_prob: 
{
  "B": 0.5954871352180898,
  "M": 0.0,
  "E": 0.0,
  "S": 0.4045128647819102
}
transition_prob: 
{
  "B": {
    "B": 0.0,
    "M": 0.11658781137718696,
    "E": 0.883412188622813,
    "S": 0.0
  },
  "M": {
    "B": 0.0,
    "M": 0.2779321774604793,
    "E": 0.7220678225395206,
    "S": 0.0
  },
  "E": {
    "B": 0.5837256220578345,
    "M": 0.0,
    "E": 0.0,
    "S": 0.41627437794216543
  },
  "S": {
    "B": 0.47243960597041273,
    "M": 0.0,
    "E": 0.0,
    "S": 0.5275603940295873
  }
}


### 3. 实现基线分词方法

In [37]:
# 正向最大匹配法（FMM）
def fmm_segment(sentence, vocab):
    max_len = max(len(word) for word in vocab)
    words = []
    i = 0
    while i < len(sentence):
        for j in range(min(max_len, len(sentence) - i), 0, -1):
            word = sentence[i:i + j]
            if word in vocab:
                words.append(word)
                i += j
                break
        else:
            words.append(sentence[i])
            i += 1
    return words

# 逆向最大匹配法（BMM）
def bmm_segment(sentence, vocab):
    max_len = max(len(word) for word in vocab)
    words = []
    i = len(sentence)
    while i > 0:
        for j in range(min(max_len, i), 0, -1):
            word = sentence[i - j:i]
            if word in vocab:
                words.insert(0, word)
                i -= j
                break
        else:
            words.insert(0, sentence[i - 1])
            i -= 1
    return words

# 双向最大匹配法（BiMM）
def bimm_segment(sentence, vocab):
    fmm_result = fmm_segment(sentence, vocab)
    bmm_result = bmm_segment(sentence, vocab)
    if len(fmm_result) < len(bmm_result):
        return fmm_result
    elif len(fmm_result) > len(bmm_result):
        return bmm_result
    else:
        if sum(len(word) for word in fmm_result) > sum(len(word) for word in bmm_result):
            return fmm_result
        else:
            return bmm_result


### 4. 实现 HMM 分词方法（使用维特比算法）

In [38]:
def viterbi(sentence, states, start_p, trans_p, emit_p):
    V = [{}]
    path = {}

    for y in states:
        V[0][y] = start_p[y] * emit_p[y].get(sentence[0], 1e-8)
        path[y] = [y]

    for t in range(1, len(sentence)):
        V.append({})
        newpath = {}

        for y in states:
            (prob, state) = max((V[t-1][y0] * trans_p[y0].get(y, 1e-8) * emit_p[y].get(sentence[t], 1e-8), y0) for y0 in states)
            V[t][y] = prob
            newpath[y] = path[state] + [y]

        path = newpath

    (prob, state) = max((V[-1][y], y) for y in states)
    return prob, path[state]

def segment(sentence):
    prob, pos_list = viterbi(sentence, states, initial_prob, transition_prob, emission_prob)
    result = []
    begin, next = 0, 0
    for i, char in enumerate(sentence):
        pos = pos_list[i]
        if pos == 'B':
            begin = i
        elif pos == 'E':
            result.append(sentence[begin:i+1])
            next = i + 1
        elif pos == 'S':
            result.append(char)
            next = i + 1
    if next < len(sentence):
        result.append(sentence[next:])
    return result

### 5. 实现评估函数

In [39]:
def evaluate(test_data, segment_func):
    TP, FP, FN = 0, 0, 0

    for line in test_data:
        line = re.sub(r'[^\w\s]', '', line).strip()
        words = line.strip().split()
        sentence = ''.join(words)
        segmented = segment_func(sentence)
        
        gold = set(words)
        pred = set(segmented)
        
        TP += len(gold & pred)
        FP += len(pred - gold)
        FN += len(gold - pred)

    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

    return precision, recall, f1


### 6. 对比结果

In [40]:

# 评估 HMM 分词方法
precision_hmm, recall_hmm, f1_hmm = evaluate(test_lines, segment)
print(f'HMM Model - Precision: {precision_hmm:.4f}, Recall: {recall_hmm:.4f}, F1-Score: {f1_hmm:.4f}')

# 评估正向最大匹配法（FMM）
precision_fmm, recall_fmm, f1_fmm = evaluate(test_lines, lambda x: fmm_segment(x, vocab))
print(f'FMM - Precision: {precision_fmm:.4f}, Recall: {recall_fmm:.4f}, F1-Score: {f1_fmm:.4f}')

# 评估逆向最大匹配法（BMM）
precision_bmm, recall_bmm, f1_bmm = evaluate(test_lines, lambda x: bmm_segment(x, vocab))
print(f'BMM - Precision: {precision_bmm:.4f}, Recall: {recall_bmm:.4f}, F1-Score: {f1_bmm:.4f}')

# 评估双向最大匹配法（BiMM）
precision_bimm, recall_bimm, f1_bimm = evaluate(test_lines, lambda x: bimm_segment(x, vocab))
print(f'BiMM - Precision: {precision_bimm:.4f}, Recall: {recall_bimm:.4f}, F1-Score: {f1_bimm:.4f}')


HMM Model - Precision: 0.4491, Recall: 0.4390, F1-Score: 0.4440
FMM - Precision: 0.9805, Recall: 0.9806, F1-Score: 0.9806
BMM - Precision: 0.9793, Recall: 0.9797, F1-Score: 0.9795
BiMM - Precision: 0.9807, Recall: 0.9805, F1-Score: 0.9806
